In [1]:
import numpy as np
import tensorflow as tf
rng = np.random

# Meta-parameters and debugging knobs
learning_rate = 0.01
training_epochs = 2000
display_step = 50

# Test data
y = np.asarray([1, 1, 1, 2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10])
num_steps = y.shape[0]

# Input data placeholders
data_in = tf.placeholder('float')
data_out = tf.placeholder('float')

# ETS params
level0 = tf.Variable(0.1 * rng.randn(), name = 'level0', dtype = tf.float32)
pace0 = tf.Variable(0.1 * rng.randn(), name = 'pace0', dtype = tf.float32)
alpha = tf.Variable(0.5, name = 'alpha', dtype = tf.float32)
beta = tf.Variable(0.1, name = 'beta', dtype = tf.float32)

# Definition of the ETS update
def update(y, level, pace):
    output = level + pace
    new_level = alpha * y + (1 - alpha) * output
    new_pace = beta * (new_level - level) + (1 - beta) * pace
    return output, new_level, new_pace

# Unrolled ETS loop
outputs = []
level, pace = level0, pace0
for time_step in range(num_steps):
    output, level, pace = update(data_in[time_step], level, pace)
    outputs.append(output)

# Mean squared error
cost = tf.reduce_sum(tf.pow(tf.pack(outputs) - data_out, 2))

# Gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Fit the data.    
    for epoch in range(training_epochs):
        sess.run(optimizer, feed_dict={data_in: y, data_out: y})

        # Display logs per epoch step
        if (epoch + 1) % display_step == 0:
            c = sess.run(cost, feed_dict={data_in: y, data_out: y})
            print "Epoch:", '%04d' % (epoch+1), \
                "cost=", "{:.9f}".format(c), \
                "level0=", sess.run(level0), \
                "pace0=", sess.run(pace0), \
                "alpha=", sess.run(alpha), \
                "beta=", sess.run(beta)

    print "Optimization Finished!"
    training_cost = sess.run(cost, feed_dict={data_in: y, data_out: y})
    print "Training cost=", training_cost, \
        "level0=", sess.run(level0), \
        "pace0=", sess.run(pace0), \
        "alpha=", sess.run(alpha), \
        "beta=", sess.run(beta)


Epoch: 0050 cost= 2.342288017 level0= 0.509123 pace0= 0.256525 alpha= 0.636935 beta= 0.751185
Epoch: 0100 cost= 2.282846212 level0= 0.656364 pace0= 0.183702 alpha= 0.652713 beta= 0.72633
Epoch: 0150 cost= 2.273410797 level0= 0.711394 pace0= 0.151309 alpha= 0.665275 beta= 0.711016
Epoch: 0200 cost= 2.271527290 level0= 0.733715 pace0= 0.138503 alpha= 0.675239 beta= 0.699067
Epoch: 0250 cost= 2.270964861 level0= 0.743322 pace0= 0.13372 alpha= 0.683217 beta= 0.689241
Epoch: 0300 cost= 2.270703554 level0= 0.74776 pace0= 0.132154 alpha= 0.689681 beta= 0.681158
Epoch: 0350 cost= 2.270549536 level0= 0.750015 pace0= 0.131855 alpha= 0.69496 beta= 0.67454
Epoch: 0400 cost= 2.270450115 level0= 0.751305 pace0= 0.132028 alpha= 0.699292 beta= 0.669133
Epoch: 0450 cost= 2.270383120 level0= 0.752136 pace0= 0.132353 alpha= 0.702859 beta= 0.664716
Epoch: 0500 cost= 2.270339966 level0= 0.752729 pace0= 0.132701 alpha= 0.705802 beta= 0.661102
Epoch: 0550 cost= 2.270309210 level0= 0.753182 pace0= 0.133024 al

**Okay, so**

That is not quite a match, but many of the components are the same. The results from R are actually:
```R
> ets(c(1, 1, 1, 2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10), model = 'AAN')
ETS(A,A,N) 

Call:
 ets(y = c(1, 1, 1, 2, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10), model = "AAN") 

  Smoothing parameters:
    alpha = 0.7199 
    beta  = 0.4639 

  Initial states:
    l = 0.7554 
    b = 0.1348 

  sigma:  0.4027

     AIC     AICc      BIC 
21.47843 28.97843 24.67371 
```
Based on a cursory look (need more digging) the model here is slightly better for reducing squared error. But I also imagine that the trend component is not particularly significant.